##  Programmation Orientée Objet , Gestion des erreurs et Gestion de fichiers

## Programmation Orientée Objet
> Programme illustrant les concepts de la POO en Python : classes, objets, héritage, encapsulation et polymorphisme.
> Cas d'usage : Système de gestion bancaire.

In [ ]:
"""
PARTIE 1 : Classes et objets : Permet de modéliser des entités réelles (ici un compte bancaire)
"""
class Client:
    # Représente un client bancaire
    def __init__(self, nom: str, email: str):
        self.nom, self.email = nom, email
    
    def __str__(self) -> str:
        return f"Client: {self.nom} ({self.email})"
    
"""
PARTIE 2 : Attributs et méthodes
"""
class CompteBancaire:
    # Représente un compte bancaire
    taux_interet: float = 0.02 # attruibut de classe
    
    def __init__(self, client: Client, solde_initial: float = 0.0):
        self.client, self.solde = client, solde_initial # attribut d'instance (client)
    
    def deposer(self, montant: float) -> None:
        self.solde = self.solde + montant
     
    def retirer(self, montant: float) -> None:
        if montant > self.solde:
            raise ValueError("Fonds inssuffisants")
        self.solde = self.solde - montant
        
    @classmethod
    def changer_taux_interet(cls, nouveau_taux: float) -> None:
        cls.taux_interet = nouveau_taux
        
    @staticmethod
    def convertir_euro_usd(montant: float, taux: float = 1.1) -> float:
        return montant * taux
    
"""
PARTIE 3 : Encapsulation : Protection des données via les getters et les setters
"""
class CompteSecure:
    def __init__(self, solde_initial: float = 0.0):
        self._solde = solde_initial # convention protected
    # pythonic way with property decorator    
    @property
    def solde(self) -> float:
        return self._solde
    
    @solde.setter
    def solde(self, valeur: float) -> None:
        if valeur < 0:
            raise ValueError("Le solde ne peut pas etre négatif")
        self._solde = valeur
        
    # non pythonic way with explicit getter and setter
    def get_solde(self) -> float:
        return self._solde
    def set_solde(self, valeur: float) -> None:
        if valeur < 0:
            raise ValueError("Le solde ne peut pas etre négatif")
        self._solde = valeur
    
"""
PARTIE 4 : Héritage
"""

class CompteEpargne(CompteBancaire):
    def appliquer_interet(self):
        self.solde = self.solde + (self.solde * self.taux_interet)
    def __str__(self):
        return f"Compte Epargne de {self.client.nom}: {self.solde:.2f}€"
    
    
"""
PARTIE 5 : Polymorphisme : Meme méthode, comportement différent
"""
class CompteCourant(CompteBancaire):
    def __str__(self) -> str:
        return f"Compte Courant de {self.client.nom}: {self.solde:.2f}€"
    
"""
PARTIE 6 : Abstraction
"""
from abc import ABC, abstractmethod

class Transaction(ABC):
    @abstractmethod
    def executer(self, compte: CompteBancaire) -> None:
        """Execute the transaction on the given account (instance method)."""
        pass

class Depot(Transaction):
    def __init__(self, montant: float):
        self.montant = montant

    def executer(self, compte: CompteBancaire) -> None:
        compte.deposer(self.montant)

class Retrait(Transaction):
    def __init__(self, montant: float):
        self.montant = montant

    def executer(self, compte: CompteBancaire) -> None:
        compte.retirer(self.montant)

    
# Démonstration en utilisant tous les concepts
# création d'un client et comptes
client = Client("Sammy", "sammy@example.com")
c_courant = CompteCourant(client, 5000)
c_epargne = CompteEpargne(client, 500)

print(c_courant)
print(c_epargne)

# Transaction avec abs
depot = Depot(350)
depot.executer(c_courant)
retrait = Retrait(20)
retrait.executer(c_epargne)

# changement du taux d'intérêt
CompteBancaire.changer_taux_interet(0.03) # en utilisant la méthode de classe
c_epargne.appliquer_interet()
print("Après transactions et application des intérêts:")
print(c_courant)
print(c_epargne)

# appel de __str__ pour démontrer le polymorphism
comptes = [c_courant, c_epargne]
for c in comptes:
    print(c)
    
# utilisation de la méthode statique
print(f"100€ en USD ($){CompteBancaire.convertir_euro_usd(100):.2f}")

# utilisation des getters et setters avec encapsulation, @property
# @property permet transformer une méthode en attribut, c'est à dire qu'on peut l'utiliser sans les parenthèses lorsque on l'appelle et on peut aussi lui attribuer une valeur lorsqu'on utilise le setter (lorsque le fonction prend quelque chose en argument)
compte_secure = CompteSecure(100)
print(f"Solde initial: {compte_secure.solde}€")
compte_secure.solde = 200
print(f"Nouveau solde: {compte_secure.solde}€")

Compte Courant de Sammy: 5000.00€
Compte Epargne de Sammy: 500.00€
Après transactions et application des intérêts:
Compte Courant de Sammy: 5350.00€
Compte Epargne de Sammy: 494.40€
Compte Courant de Sammy: 5350.00€
Compte Epargne de Sammy: 494.40€
100€ en USD ($)110.00
Solde initial: 100€
Nouveau solde: 200€


## Comment @property foctionne en Python

In [3]:
# Getter simple (lecture dynamique)

class Rectangle:
    def __init__(self, largeur: float, hauteur: float):
        self.largeur, self.hauteur = largeur, hauteur
    @property
    def aire(self):
        return self.largeur * self.hauteur
    
# Utilisation
r = Rectangle(4,6)
print(r.aire) # 24, pas besoin d'écire r.aire()

24


In [4]:
# Getter + Setter

class Compte:
    def __init__(self, solde):
        self._solde = solde  # attribut protégé

    @property
    def solde(self):
        """Getter : lire le solde"""
        return self._solde

    @solde.setter
    def solde(self, nouveau_solde):
        """Setter : modifier avec validation"""
        if nouveau_solde < 0:
            raise ValueError("Le solde ne peut pas être négatif")
        self._solde = nouveau_solde


# --- Utilisation ---
c = Compte(100)
print(c.solde)   # 100 → passe par le getter
c.solde = 200    # OK → passe par le setter
# c.solde = -50  #  Erreur

100


In [ ]:
# Getter + setter + Deleter (Pour controler la supperession d'un attribut)
class Employe:
    def __init__(self, nom: str):
        self._nom = nom
        
    @property
    def nom(self):
        return self._nom
    
    @nom.setter
    def nom(self, nouveau_nom):
        if not nouveau_nom.strip():
            raise ValueError("Nom invlide")
        self._nom = nouveau_nom
    
    @nom.deleter
    def nom(self):
        print("Suppression du nom...")
        del self._nom
# Utilisation
e = Employe("Emy")
print(e.nom) # getter
e.nom = "Sam" # setter
del e.nom       #deleter

In [5]:
class Cercle:
    def __init__(self, rayon):
        self.rayon = rayon

    @property
    def diametre(self):
        return self.rayon * 2

    @property
    def aire(self):
        import math
        return math.pi * (self.rayon ** 2)


# --- Utilisation ---
c = Cercle(10)
print(c.diametre)  # 20
print(c.aire)      # 314.15...
c.rayon = 5
print(c.aire)      # 78.5... (recalculé automatiquement)

20
314.1592653589793
78.53981633974483


In [5]:
# Comment l'attribut de classe est protégé contre la modification directe
class Product:
    # Attribut de classe : taux de promotion commun à tous les produits
    promotion_rate: float = 0.20  # 20% de réduction
    tous_les_produits = []  # Liste pour suivre toutes les instances de produits
    def __init__(self, name: str, base_price: float):
        print(f"Création du produit {name} avec un prix de {base_price}€")
        self.name = name
        self.base_price = base_price  # Prix de base (attribut d'instance)
        Product.tous_les_produits.append(self)  # Ajoute l'instance à la liste
        print(f"Processus terminé pour {name}\n")
    # représentation textuelle de l'objet, un peu plus user friendly
    # def __str__(self) -> str:
    #     return f"Produit: {self.name}, Prix de base: {self.base_price:.2f}€, Taux de promotion: {self.promotion_rate * 100}%"
    
    # représentation officielle de l'objet, utile pour le debug
    def __repr__(self) -> str:
        return f"Product(name={self.name}, base_price={self.base_price}, promotion_rate={self.promotion_rate})"

    def get_promotional_price(self) -> float:
        # Calcule le prix promotionnel en utilisant l'attribut de classe.
        return self.base_price * (1 - self.promotion_rate)

    @classmethod
    def get_promotion_rate(cls) -> float:
        # Méthode de classe pour accéder au taux de promotion.
        return cls.promotion_rate
    
    @classmethod
    def get_nouvelle_promotion_rate(cls, nouveau_taux: float) -> None:
        cls.promotion_rate = nouveau_taux # Méthode de classe pour modifier le taux de promotion.

# Création de plusieurs produits
p1 = Product("T-Shirt", 29.99)
p2 = Product("Jeans", 59.99)
p3 = Product("Chaussures", 89.99)

# Accès à l'attribut de classe via les objets
print(f"Taux de promotion actuel : {Product.get_promotion_rate() * 100}%")  # 20%
print(f"Prix promo {p1.name} : {p1.get_promotional_price():.2f}€")  # 23.99€
# supposons que les temps ont changé et on veut modifier le taux de promotion pour certains produits
# Product.promotion_rate = 0.30  # Mauvaise pratique : modification directe de l'attribut de classe
# ce qu'il faut faire
p2.promotion_rate = 0.30  # Crée un attribut d'instance, ne modifie pas l'attribut de classe
print(f"Taux de promotion après modification via p2 : {p2.promotion_rate * 100}%")  # 30%
print(f"Prix promo {p2.name} avec nouveau taux : {p2.get_promotional_price():.2f}€")  # 41.99€
# Vérification : l'attribut de classe n'a pas changé
print(f"Taux de promotion après accès : {Product.promotion_rate * 100}%")  # Toujours 20%

# Maintenant, si on veut vraiment changer le taux de promotion pour tous les produits
Product.get_nouvelle_promotion_rate(0.25)  # Change le taux de promotion à 25%
print(f"Nouveau taux de promotion via la classe : {Product.get_promotion_rate() * 100}%")  # 25%
print(f"Prix promo {p3.name} avec nouveau taux : {p3.get_promotional_price():.2f}€")  # 67.49€

print("\nListe de tous les produits créés:")
for produit in Product.tous_les_produits:
    print(produit)
    

Création du produit T-Shirt avec un prix de 29.99€
Processus terminé pour T-Shirt

Création du produit Jeans avec un prix de 59.99€
Processus terminé pour Jeans

Création du produit Chaussures avec un prix de 89.99€
Processus terminé pour Chaussures

Taux de promotion actuel : 20.0%
Prix promo T-Shirt : 23.99€
Taux de promotion après modification via p2 : 30.0%
Prix promo Jeans avec nouveau taux : 41.99€
Taux de promotion après accès : 20.0%
Nouveau taux de promotion via la classe : 25.0%
Prix promo Chaussures avec nouveau taux : 67.49€

Liste de tous les produits créés:
Product(name=T-Shirt, base_price=29.99, promotion_rate=0.25)
Product(name=Jeans, base_price=59.99, promotion_rate=0.3)
Product(name=Chaussures, base_price=89.99, promotion_rate=0.25)


In [ ]:
# Méthodes statiques et de classe
class Item:
    @staticmethod
    def my_static_method():
        """
        Correction : "Cette méthode doit être liée logiquement à la classe, mais ne dépend ni de la classe ni des instances.
        Elle ne modifie pas l'état de la classe ou des instances."
        """
        pass

    @classmethod
    def my_class_method(cls):
        """
        "Cette méthode agit sur la classe elle-même (via `cls`).
        Elle peut modifier des attributs de classe ou être utilisée comme factory method pour créer des instances."
        """
        pass

In [8]:
# comment l'héritage fonctionne
class CompteBancaire:
    def __init__(self, solde_initial):
        self.solde = solde_initial

    def retirer(self, montant):
        if montant > self.solde:
            raise ValueError("Solde insuffisant")
        self.solde -= montant

class CompteEpargne(CompteBancaire):
    def __init__(self, solde_initial, taux_interet):
        super().__init__(solde_initial)  # Appelle le constructeur de CompteBancaire
        self.taux_interet = taux_interet

    def retirer(self, montant):
        if montant > self.solde / 2:  # Limite de retrait à 50% du solde
            raise ValueError("Retrait limité à 50% du solde")
        super().retirer(montant)  # Appelle la méthode retirer de CompteBancaire

# Utilisation
compte = CompteEpargne(1000, 0.05)
compte.retirer(400)  # OK
compte.retirer(600)  # ValueError: Retrait limité à 50% du solde

ValueError: Retrait limité à 50% du solde

In [ ]:
# comment une classe peut contenir une autre classe comme attribut
class Moteur:
    # classe représentant un moteur
    def __init__(self, type_carburant: str, puissance_cv: int):
        self.type_carburant = type_carburant
        self.puissance_cv = puissance_cv
        self.est_allume = False
        
    def demarrer(self):
        if not self.est_allume:
            print("Vroum vroum..... le moteur démarre...")
            self.est_allume = True
        else:
            print("Le moteur est déjà allumé.")
            
    def eteindre(self):
        if self.est_allume:
            print("Le moteur est éteint...")
            self.est_allume = False
    
class Voiture:
    # classe représentant une voiture
    def __init__(self, marque :str, modele: str, moteur: Moteur):
        print("Fabrication de voiture en cours...")
        self.marque = marque
        self.modele = modele
        self.moteur = moteur
        
    def conduire(self):
        print(f"Conduite de la {self.marque} modèle {self.modele}")
        self.moteur.demarrer()
        
    def __str__(self):
        return f"{self.marque} {self.modele} avec un moteur de {self.moteur.puissance_cv} CV fonctionnant à l'{self.moteur.type_carburant}"
        
# Output

# création des différents moteurs
moteur_v10 = Moteur(type_carburant="Essence", puissance_cv=1200)
moteur_v8 = Moteur(type_carburant="Essence", puissance_cv=500)
moteur_electrique= Moteur(type_carburant="Electricité", puissance_cv=100)
# création des voitures

ma_peugot = Voiture(marque="Peugot", modele="2007", moteur=moteur_v8)
print(ma_peugot)
ma_peugot.conduire()

ma_tesla = Voiture(marque="Tesla", modele="S", moteur=moteur_electrique)
print(ma_tesla)
ma_tesla.conduire()
ma_BM = Voiture(marque="BMW", modele="w11", moteur=moteur_v8)

Fabrication de voiture en cours...
Peugot 2007 avec un moteur de 500 CV fonctionnant à l'Essence
Conduite de la Peugot modèle 2007
Vroum vroum..... le moteur démarre...
Fabrication de voiture en cours...
Tesla S avec un moteur de 100 CV fonctionnant à l'Electricité
Conduite de la Tesla modèle S
Vroum vroum..... le moteur démarre...
Fabrication de voiture en cours...


# Gestion des erreurs